In [1]:
import spacy
import pandas as pd

In [2]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("The quick brown fox jumps over the lazy dog.")

In [3]:
token_list = [[token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
               token.shape_, token.is_alpha, token.is_stop] for token in doc]

df = pd.DataFrame(columns=['text', 'lemma_', 'pos_', 'tag_', 'dep_', 'shape_', 'is_alpha', 'is_stop'], 
                  data=token_list)

df

,text,lemma_,pos_,tag_,dep_,shape_,is_alpha,is_stop
0,The,the,DET,DT,det,Xxx,True,True
1,quick,quick,ADJ,JJ,amod,xxxx,True,False
2,brown,brown,ADJ,JJ,amod,xxxx,True,False
3,fox,fox,NOUN,NN,nsubj,xxx,True,False
4,jumps,jump,VERB,VBZ,ROOT,xxxx,True,False
5,over,over,ADP,IN,prep,xxxx,True,True
6,the,the,DET,DT,det,xxx,True,True
7,lazy,lazy,ADJ,JJ,amod,xxxx,True,False
8,dog,dog,NOUN,NN,pobj,xxx,True,False
9,.,.,PUNCT,.,punct,.,False,False


In [4]:
spacy.explain('DET')

'determiner'

In [5]:
print("Pipeline:", nlp.pipe_names)
doc = nlp("I was reading the paper.")
token = doc[0]  # 'I'
print(token.morph)
print(token.morph.get("PronType"))

Pipeline: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Case=Nom|Number=Sing|Person=1|PronType=Prs
['Prs']


In [10]:
from spacy.lang.en import English

nlp = spacy.blank("en")  # just the language with no pipeline
nlp.add_pipe("sentencizer")
# nlp.add_pipe("tagger")
# nlp.add_pipe("entity_linker")
print(nlp.analyze_pipes(pretty=True))


============================= Pipeline Overview =============================

#   Component     Assigns               Requires   Scores    Retokenizes
-   -----------   -------------------   --------   -------   -----------
0   sentencizer   token.is_sent_start              sents_f   False      
                  doc.sents                        sents_p              
                                                   sents_r              

✔ No problems found.
{'summary': {'sentencizer': {'assigns': ['token.is_sent_start', 'doc.sents'], 'requires': [], 'scores': ['sents_f', 'sents_p', 'sents_r'], 'retokenizes': False}}, 'problems': {'sentencizer': []}, 'attrs': {'doc.sents': {'assigns': ['sentencizer'], 'requires': []}, 'token.is_sent_start': {'assigns': ['sentencizer'], 'requires': []}}}


In [8]:
doc = nlp("This is a sentence. This is another sentence.")
for sent in doc.sents:
    print(sent.text)

This is a sentence.
This is another sentence.


In [6]:
from spacy import displacy

In [11]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Hello World!")
displacy.serve(doc, style="dep")


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [9]:
spacy.explain('npadvmod')

'noun phrase as adverbial modifier'

# Article summary

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

In [15]:
import requests
from bs4 import BeautifulSoup

def scrap_text(url, tag='p', **kwargs):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    text = ' '.join(map(lambda p: p.text, soup.find_all(tag, **kwargs)))
    return text

In [13]:
article_url = "https://jamesclear.com/why-facts-dont-change-minds"

In [17]:
article = scrap_text(article_url)
article

'The economist J.K. Galbraith once wrote, “Faced with a choice between changing one’s mind and proving there is no need to do so, almost everyone gets busy with the proof.” Leo Tolstoy was even bolder: “The most difficult subjects can be explained to the most slow-witted man if he has not formed any idea of them already; but the simplest thing cannot be made clear to the most intelligent man if he is firmly persuaded that he knows already, without a shadow of doubt, what is laid before him.” What’s going on here? Why don’t facts change our minds? And why would someone continue to believe a false or inaccurate idea anyway? How do such behaviors serve us?\n Humans need a reasonably accurate view of the world in order to survive. If your model of reality is wildly different from the actual world, then you struggle to take effective actions each day. 1 However, truth and accuracy are not the only things that matter to the human mind. Humans also seem to have a deep desire to belong. In Ato

In [19]:
nlp = spacy.load("en_core_web_sm")

In [20]:
doc = nlp(article)
doc

The economist J.K. Galbraith once wrote, “Faced with a choice between changing one’s mind and proving there is no need to do so, almost everyone gets busy with the proof.” Leo Tolstoy was even bolder: “The most difficult subjects can be explained to the most slow-witted man if he has not formed any idea of them already; but the simplest thing cannot be made clear to the most intelligent man if he is firmly persuaded that he knows already, without a shadow of doubt, what is laid before him.” What’s going on here? Why don’t facts change our minds? And why would someone continue to believe a false or inaccurate idea anyway? How do such behaviors serve us?
 Humans need a reasonably accurate view of the world in order to survive. If your model of reality is wildly different from the actual world, then you struggle to take effective actions each day. 1 However, truth and accuracy are not the only things that matter to the human mind. Humans also seem to have a deep desire to belong. In Atomi

In [21]:
len(list(doc.sents))

161

In [64]:
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import defaultdict
from heapq import nlargest
_stopwords = set(list(STOP_WORDS) + list(punctuation) + ['’', "'", '"', '\n'])

In [61]:
freq = defaultdict(int)

for word in doc:
    if word.lemma_ not in _stopwords:
        freq[word.lemma_] += 1

In [63]:
ranking = defaultdict(int)

for i, sent in enumerate(doc.sents):
    for w in doc:
        if w.lemma_ in freq:
            ranking[i] += freq[w]

In [76]:
sents_idx = nlargest(4, ranking, key=ranking.get)
' '.join([str([*doc.sents][i]) for i in sorted(sents_idx)])

'The economist J.K. Galbraith once wrote, “Faced with a choice between changing one’s mind and proving there is no need to do so, almost everyone gets busy with the proof.” Leo Tolstoy was even bolder: “The most difficult subjects can be explained to the most slow-witted man if he has not formed any idea of them already; but the simplest thing cannot be made clear to the most intelligent man if he is firmly persuaded that he knows already, without a shadow of doubt, what is laid before him.” What’s going on here? Why don’t facts change our minds?'